In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import geopandas
import cartopy.feature as cfeature
from datetime import datetime, timedelta
from cartopy import crs as ccrs
from shapely import geometry

In [7]:
ground_truth_df = geopandas.read_file('../references/qgis_verification/Sep2019/Sep2019_groundtruth.shp')

In [8]:
ground_truth_df['lat'] = ground_truth_df.geometry.y
ground_truth_df['lon'] = ground_truth_df.geometry.x

In [9]:
ground_truth_df.rename(columns={"Date": "datestamp"}, inplace=True)


In [10]:
ground_truth_df['startdate'] = '0'
ground_truth_df['enddate'] = ground_truth_df['datestamp']

for index, row in ground_truth_df.iterrows():
    end_day = int(row['enddate'].split('/')[2])
    end_month = int(row['enddate'].split('/')[1])
    end_year = int(row['enddate'].split('/')[0])
    ground_truth_df.loc[index, 'end_date'] = datetime(end_year, end_month, end_day)
            
    if row['startdate'] == '0':
        ground_truth_df.loc[index, 'start_date'] = datetime(end_year, end_month, end_day)
    else:
        ground_truth_df.loc[index, 'start_date'] = datetime(end_year, start_month, start_day)

In [11]:
ground_truth_df['period'] = ground_truth_df['end_date'] - ground_truth_df['start_date']

In [12]:
ground_truth_gjson = ground_truth_df.copy()
ground_truth_gjson

,datestamp,geometry,lat,lon,startdate,enddate,end_date,start_date,period
0,2019/09/01,POINT (102.15084 0.42880),0.428803,102.150839,0,2019/09/01,2019-09-01,2019-09-01,0 days
1,2019/09/01,POINT (102.15204 0.42568),0.425679,102.152041,0,2019/09/01,2019-09-01,2019-09-01,0 days
2,2019/09/01,POINT (102.40703 -0.12635),-0.126350,102.407027,0,2019/09/01,2019-09-01,2019-09-01,0 days
3,2019/09/01,POINT (102.39958 -0.12563),-0.125629,102.399577,0,2019/09/01,2019-09-01,2019-09-01,0 days
4,2019/09/01,POINT (102.41183 -0.12611),-0.126109,102.411833,0,2019/09/01,2019-09-01,2019-09-01,0 days
5,2019/09/01,POINT (102.49859 0.88686),0.886864,102.498591,0,2019/09/01,2019-09-01,2019-09-01,0 days
6,2019/09/01,POINT (102.61154 -0.36307),-0.363071,102.611544,0,2019/09/01,2019-09-01,2019-09-01,0 days
7,2019/09/01,POINT (102.65576 -0.39576),-0.395755,102.655764,0,2019/09/01,2019-09-01,2019-09-01,0 days
8,2019/09/01,POINT (103.94026 -1.25146),-1.251460,103.940258,0,2019/09/01,2019-09-01,2019-09-01,0 days
9,2019/09/01,POINT (103.98770 -1.28780),-1.287797,103.987699,0,2019/09/01,2019-09-01,2019-09-01,0 days


In [13]:
ground_truth_gjson = ground_truth_gjson.to_crs({'init': 'epsg:3857'})
ground_truth_gjson.drop(columns=['startdate', 'enddate'], inplace=True)

In [14]:
ground_truth_gjson.to_csv('../references/qgis_verification/Sep2019/Sep2019_groundtruth.csv')

In [ ]:
hs_df_clipped = geopandas.overlay(clip_area, hs_df, how='intersection')

NameError: name 'hs_df_clipped' is not defined